# Stratified K 폴드



stratified K fold는 불균형한 분포도를 가진 레이블(결정 클래스) 데이터 집합을 위한 K fold 방식입니다. <br>
**원본 데이터의 특징을 나타내는 불균형한 분포를 학습/테스트 세트에도 유지하는 것이 중요합니다.**

In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()
#value 0,1,2 모두 50개로 동일합니다.

2    50
1    50
0    50
Name: label, dtype: int64

### 일반 Kfold 방식

In [ ]:
kfold=KFold(n_splits=3)
n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    #print('##교차 검증:{0}'.format(n_iter))
    #print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    #print('검증 레이블 데이터 분포:\n', label_test.value_counts())

### Stratified 방식

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    # train_index에서 나온 인덱스는 train data set이고, 반대도 마찬가지입니다.
    #print('##교차 검증:{0}'.format(n_iter))
    #print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    #print('검증 레이블 데이터 분포:\n', label_test.value_counts())

### Stratified 방식 가지고 예측

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=156)

skf = StratifiedKFold(n_splits=3)
n_iter = 0
cv_accuracy=[]
features = iris.data
label = iris.target


for train_index, test_index in skf.split(features, label):

    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)

    #반복 시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    #print('\n#{0} 교차 검증 정확도:{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    #print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)

#print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
#print('##평균 검증 정확도:', np.mean(cv_accuracy))

### cross_val_score()
이 친구는 교차 검증을 좀 더 편리하게 수행할 수 있게 해주는 API입니다. <br>
kfold로 데이터를 학습하고 예측하는 코드를 보면 <br>
1. 폴드 세트를 설정
2. for 루프에서 반복적으로 학습 및 데스트 데이터의 인덱스를 추출
3. 반복적으로 학습과 예측을 수행하고 예측 성능을 반환
<br>
ex) corss_val_score(estimator, X, y=None, scoring=None, cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch='2*n_jobs') <br>
여기서 중요한 것은 estimator(classifier or regressor), X(feature dataset), y(label dataset), scoring(예측 성능 평가 지표), cv(교차 검증 폴드 수)

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

#성능 지표는 정확도(accuracy),교차검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('\n## 교차 검증별 정확도:', scores)
print('##평균 검증 정확도:', np.mean(scores))


## 교차 검증별 정확도: [0.98039216 0.92156863 0.97916667]
##평균 검증 정확도: 0.960375816993464


### cross_val_score이 내부적으로 stratified kfold를 사용하기에 accuracy값이 같다.